In [ ]:
import os
import csv
from collections import deque

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib import collections  as mc
%matplotlib inline

%load_ext autoreload
%autoreload 1

%aimport models
%aimport utils

In [ ]:
our_recs = sorted(utils.enumerate_recording_paths('../dataset/raw/*record*'))
our_recs += sorted(utils.enumerate_recording_paths('../dataset/raw/benni*/*record*'))
recs = [models.Recording(p) for p in our_recs]

In [ ]:
# plt.plot(r.RightFootSensor['arrival_ts'], r.RightFootSensor['event_ts'])

In [ ]:
def step_gaps(r):
    zipped_times = zip(r.RightFootSensor['arrival_ts'], r.RightFootSensor['duration'] * 1e-3)
    right_lines = [[(arr - dur, 0), (arr, 0)] for arr, dur in zipped_times]

    zipped_times = zip(r.LeftFootSensor['arrival_ts'], r.LeftFootSensor['duration'] * 1e-3)
    left_lines = [[(arr - dur, 0), (arr, 0)] for arr, dur in zipped_times]

    # combine all lines and sort by starting point
    all_lines = sorted(right_lines + left_lines, key=lambda x: x[0][0])
    gaps_per_rec = []
    for i, prev_line in enumerate(all_lines):
        prev_end = prev_line[1][0]
        try:
            next_start = all_lines[i + 1][0][0]
        except IndexError:
            break
        if prev_end < next_start:
            gaps_per_rec.append((prev_end, next_start))
            
    gaps = np.array(gaps_per_rec)
    gap_diffs = (gaps[:, 1] - gaps[:, 0])
    wo_sm_gap_diffs = gap_diffs[gap_diffs < 2.]
    wo_sm_gap_diffs = wo_sm_gap_diffs[wo_sm_gap_diffs >= 0]
    gap_diffs -= 0.75
    w_sm_gap_diffs = gap_diffs[gap_diffs < 2.]
    w_sm_gap_diffs = w_sm_gap_diffs[w_sm_gap_diffs >= 0]
    return wo_sm_gap_diffs, w_sm_gap_diffs

In [ ]:
from itertools import chain
from string import ascii_uppercase

all_gaps_wo = []
all_gaps_w = []
for r in recs:
    wo_sm, w_sm = step_gaps(r)
    all_gaps_wo.append(wo_sm)
    all_gaps_w.append(w_sm)

figsize = (10,4)
plt.figure(figsize=figsize)
plt.title('Step gap distribution for gap durations < 2 seconds; $step\_margin=0.0$')
_ = plt.boxplot(all_gaps_wo)
# _ = plt.xticks(*zip(*[(i + 1, os.path.split(r.path)[-1]) for i, r in enumerate(recs)]),
#                rotation='vertical')
_ = plt.xticks(*zip(*[(i + 1, f'Recording {ascii_uppercase[i]} — {g.shape[0]} gaps') for i, g in enumerate(all_gaps_wo)]), rotation=-45, ha='left', va='top')
# plt.xlabel('Recording name')
plt.ylabel('Gap duration')
plt.tight_layout()
plt.savefig('figures/gap_distribution_wo.png')

plt.figure(figsize=figsize)
plt.title('Step gap distribution for gap durations < 2 seconds; $step\_margin=0.75$')
_ = plt.boxplot(all_gaps_w)
# _ = plt.xticks(*zip(*[(i + 1, os.path.split(r.path)[-1]) for i, r in enumerate(recs)]),
#                rotation='vertical')
_ = plt.xticks(*zip(*[(i + 1, f'Recording {ascii_uppercase[i]} — {g.shape[0]} gaps') for i, g in enumerate(all_gaps_w)]), rotation=-45, ha='left', va='top')
# plt.xlabel('Recording name')
plt.ylabel('Gap duration')
plt.tight_layout()
plt.savefig('figures/gap_distribution_w.png')
all_gaps_flat = list(chain(*all_gaps_w, *all_gaps_wo))
print('Step gap duration mean',np.mean(all_gaps_flat))

In [ ]:
def plot_step_durations(r, step_margin=0.):
    prop_cycle = plt.rcParams['axes.prop_cycle']
    colors = prop_cycle.by_key()['color']

    fig, ax = plt.subplots(figsize=(16, 9))
    for i, dim in enumerate(('val_x', 'val_y', 'val_z')):
        plt.plot(r.Accelerometer['arrival_ts'],
                 r.Accelerometer[dim], 'x', label=dim)
    
    zipped_times = zip(r.RightFootSensor['arrival_ts'], r.RightFootSensor['duration'] * 1e-3)
    right_lines = [[(arr - dur - step_margin, 5), (arr, 5)] for arr, dur in zipped_times]

    zipped_times = zip(r.LeftFootSensor['arrival_ts'], r.LeftFootSensor['duration'] * 1e-3)
    left_lines = [[(arr - dur - step_margin, 5), (arr, 5)] for arr, dur in zipped_times]
        
    lc = mc.LineCollection(right_lines, linewidths=200, colors=colors[3], zorder=10)
    ax.add_collection(lc)
    lc = mc.LineCollection(left_lines, linewidths=200, colors=colors[4], zorder=10)
    ax.add_collection(lc)
    ax.autoscale()
    plt.title(r.path)
    plt.legend()
    plt.show()
    plt.close()

In [ ]:
for r in recs:
    plot_step_durations(r, .75)

In [ ]:
%aimport data_mappers
WC = data_mappers.Window_Creator
recs = sorted(utils.enumerate_recording_paths('../dataset'), key=lambda r: os.path.split(r)[-1])
source = WC(recs[-9:])

In [ ]:
for i in range(0, source.labels.shape[0], 1000):
    plt.figure(figsize=(16,4))
    plt.plot(source.accelerometer[i, :, :], 'x', label=source.labels[i] > .5)
    plt.legend()